In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [4]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

# Load the CSV file
voter_df = spark.read.format('csv').options(Header=True).load('file:///home/talentum/test-jupyter/P3/M4/sm16/Dataset/DallasCouncilVoters.csv.gz')

voter_df = voter_df.filter(voter_df['VOTER_NAME'].isNotNull())

# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))

# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn('first_name', voter_df.splits.getItem(0))

def getMiddleName(names):
# Return a space separated string of names\
    return ' '.join(names[1:-1])

# Define the method as a UDF
udfMiddle = F.udf(getMiddleName, StringType())

# Create a new column using your UDF
voter_df = voter_df.withColumn('middle_name', udfMiddle(F.col('splits')))

# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn('last_name', voter_df.splits.getItem(F.size('splits') - 1))

voter_df=voter_df.drop('splits', 'VOTER_NAME')
# Show the DataFrame
voter_df.show()

+----------+-------------+----------+-----------+---------+
|      DATE|        TITLE|first_name|middle_name|last_name|
+----------+-------------+----------+-----------+---------+
|02/08/2017|Councilmember|  Jennifer|         S.|    Gates|
|02/08/2017|Councilmember|    Philip|         T.| Kingston|
|02/08/2017|        Mayor|   Michael|         S.| Rawlings|
|02/08/2017|Councilmember|      Adam|           |  Medrano|
|02/08/2017|Councilmember|     Casey|           |   Thomas|
|02/08/2017|Councilmember|   Carolyn|       King|   Arnold|
|02/08/2017|Councilmember|     Scott|           |   Griggs|
|02/08/2017|Councilmember|        B.|       Adam|  McGough|
|02/08/2017|Councilmember|       Lee|           | Kleinman|
|02/08/2017|Councilmember|     Sandy|           |  Greyson|
|02/08/2017|Councilmember|  Jennifer|         S.|    Gates|
|02/08/2017|Councilmember|    Philip|         T.| Kingston|
|02/08/2017|        Mayor|   Michael|         S.| Rawlings|
|02/08/2017|Councilmember|      Adam|   